# Nepal Database Project SQLite

In [1]:
# import libary
import sqlite3

In [2]:
# connecting to the Nepal Earthquake Database
%reload_ext sql
%sql sqlite:///nepal.sqlite

In [3]:
%%sql
-- Inspecting the database 
SELECT *
FROM sqlite_schema

 * sqlite:///nepal.sqlite
Done.


[('table', 'id_map', 'id_map', 2, 'CREATE TABLE "id_map" (\n"household_id" INTEGER,\n  "building_id" INTEGER,\n  "vdcmun_id" INTEGER,\n  "district_id" INTEGER\n)'),
 ('index', 'ix_id_map_household_id', 'id_map', 3, 'CREATE INDEX "ix_id_map_household_id"ON "id_map" ("household_id")'),
 ('table', 'building_structure', 'building_structure', 2032, 'CREATE TABLE "building_structure" (\n"building_id" INTEGER,\n  "count_floors_pre_eq" INTEGER,\n  "count_floors_post_eq" INTEGER,\n  "age_building" IN ... (198 characters truncated) ... or_type" TEXT,\n  "other_floor_type" TEXT,\n  "position" TEXT,\n  "plan_configuration" TEXT,\n  "condition_post_eq" TEXT,\n  "superstructure" TEXT\n)'),
 ('index', 'ix_building_structure_building_id', 'building_structure', 2033, 'CREATE INDEX "ix_building_structure_building_id"ON "building_structure" ("building_id")'),
 ('table', 'building_damage', 'building_damage', 12302, 'CREATE TABLE "building_damage" (\n"building_id" INTEGER,\n  "damage_overall_collapse" TEXT,\n  "damage_overall_leaning" TEXT,\n  "damage_overall_adja ... (2923 characters truncated) ... ndslide" INTEGER,\n  "has_geotechnical_risk_rock_fall" INTEGER,\n  "has_geotechnical_risk_flood" INTEGER,\n  "has_geotechnical_risk_other" INTEGER\n)'),
 ('index', 'ix_building_damage_building_id', 'building_damage', 12305, 'CREATE INDEX "ix_building_damage_building_id"ON "building_damage" ("building_id")'),
 ('table', 'household_demographics', 'household_demographics', 31601, 'CREATE TABLE "household_demographics" (\n"household_id" INTEGER,\n  "gender_household_head" TEXT,\n  "age_household_head" REAL,\n  "caste_household"  ... (8 characters truncated) ...  "education_level_household_head" TEXT,\n  "income_level_household" TEXT,\n  "size_household" REAL,\n  "is_bank_account_present_in_household" REAL\n)'),
 ('index', 'ix_household_demographics_household_id', 'household_demographics', 31602, 'CREATE INDEX "ix_household_demographics_household_id"ON "household_demographics" ("household_id")')]

In [5]:
%%sql
-- Getting the names of tables in the Database.
SELECT name
FROM sqlite_schema
WHERE type ="table"

 * sqlite:///nepal.sqlite
Done.


name
id_map
building_structure
building_damage
household_demographics


Now that i know the tables available, I'll be treating some questions to extract valuable insights from the tables provided.

### What are the different types of building structures present in the database?

In [11]:
%%sql
-- Getting the types of building structures present in the database 
SELECT DISTINCT superstructure
FROM building_structure;


 * sqlite:///nepal.sqlite
Done.


superstructure
"Stone, mud mortar"
Bamboo
Stone
Other
"Brick, cement mortar"
Adobe/mud
"Brick, mud mortar"
Timber
"Stone, cement mortar"
"RC, non-engineered"


 ### What is the distribution of building structures across different regions or areas?

In [14]:
%%sql
-- Getting the distribution of building structures across different regions
SELECT m.district_id, bs.superstructure, COUNT(*) AS count
FROM building_structure bs
JOIN id_map AS m
ON bs.building_id = m.building_id
GROUP BY m.district_id, bs.superstructure;

 * sqlite:///nepal.sqlite
Done.


district_id,superstructure,count
1,Adobe/mud,1640
1,Bamboo,279
1,"Brick, cement mortar",106
1,"Brick, mud mortar",6
1,Other,66
1,"RC, engineered",5
1,"RC, non-engineered",40
1,Stone,43
1,"Stone, cement mortar",126
1,"Stone, mud mortar",33406


### What is the average age of buildings in the database?

In [16]:
%%sql
-- Getting the average age of buildings in the database
SELECT ROUND(AVG(age_building)) AS average_age_of_buildings
FROM building_structure;

 * sqlite:///nepal.sqlite
Done.


average_age_of_buildings
27.0


### How many buildings in the database have been damaged?


In [17]:
%%sql
-- Getting the buildings in the database have been damaged
SELECT COUNT(DISTINCT building_id) AS damaged_buildings_count
FROM building_damage;

 * sqlite:///nepal.sqlite
Done.


damaged_buildings_count
234835


###  Is there any correlation between building structure and building damage?

In [18]:
%%sql
-- Getting the correlation between building structure and building damage
SELECT bs.superstructure, COUNT(*) AS total_buildings, COUNT(bd.building_id) AS damaged_buildings
FROM building_structure bs
LEFT JOIN building_damage bd 
ON bs.building_id = bd.building_id
GROUP BY bs.superstructure;


 * sqlite:///nepal.sqlite
Done.


superstructure,total_buildings,damaged_buildings
Adobe/mud,14796,14796
Bamboo,703,703
"Brick, cement mortar",5929,5929
"Brick, mud mortar",4897,4897
Other,436,436
"RC, engineered",2122,2122
"RC, non-engineered",3289,3289
Stone,2943,2943
"Stone, cement mortar",1376,1376
"Stone, mud mortar",196841,196841


###  Which buildings have a higher damage severity level than the average severity level of all buildings?

In [17]:
%%sql 
SELECT building_id, damage_grade
FROM building_damage
WHERE damage_grade > (
    SELECT AVG(damage_grade)
    FROM building_damage
    
)
limit 25;

 * sqlite:///nepal.sqlite
Done.


building_id,damage_grade
1,Grade 3
2,Grade 5
3,Grade 2
4,Grade 2
5,Grade 1
6,Grade 2
7,Grade 5
8,Grade 2
9,Grade 3
10,Grade 3


###  Which region have the highest percentage of buildings with severe damage?

In [19]:
%%sql
WITH region_damage_percentage AS (
  SELECT district_id, COUNT(*) AS total_buildings,
         SUM(CASE WHEN damage_grade = 'Grade 5' THEN 1 ELSE 0 END) AS severe_buildings,
         CAST(SUM(CASE WHEN damage_grade = 'Grade 5' THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) AS percentage_severe
  FROM building_damage
  JOIN id_map 
  ON building_damage.building_id = id_map.building_id
  GROUP BY district_id
)
SELECT district_id
FROM region_damage_percentage
WHERE percentage_severe = (
    SELECT MAX(percentage_severe)
    FROM region_damage_percentage
);


 * sqlite:///nepal.sqlite
Done.


district_id
4


###  How many households fall within each income bracket in region (district_id = 4)?

In [27]:
%%sql
WITH region_income_bracket AS (
  SELECT m.district_id, 
         CASE 
            WHEN income_level_household = "Rs. 10 thousand" THEN 'Low'
            WHEN income_level_household = "Rs. 20-30 thousand" THEN 'Medium'
            WHEN income_level_household = "Rs. 30-50 thousand" THEN 'High'
            ELSE 'Unknown'
         END AS income_bracket
  FROM household_demographics AS h
  JOIN id_map AS m
  ON m.household_id = h.household_id
  WHERE m.district_id = 4
)
SELECT income_bracket, COUNT(*) AS household_count
FROM region_income_bracket
GROUP BY income_bracket;


 * sqlite:///nepal.sqlite
Done.


income_bracket,household_count
High,3044
Low,41221
Medium,7963
Unknown,23655


### Determine the most common technical solution proposed for each type of damage?

In [34]:
%%sql
WITH most_common_solution AS (
    SELECT 
        area_assesed,
        technical_solution_proposed,
        ROW_NUMBER() OVER (PARTITION BY area_assesed ORDER BY COUNT(*) DESC) AS rn
    FROM building_damage
    GROUP BY area_assesed, technical_solution_proposed
)
SELECT area_assesed, technical_solution_proposed
FROM most_common_solution
WHERE rn = 1;


 * sqlite:///nepal.sqlite
Done.


area_assesed,technical_solution_proposed
None,None
Both,Reconstruction
Building removed,Reconstruction
Exterior,Reconstruction
Interior,Reconstruction
Not able to inspect,Reconstruction


### What is the average age of buildings for each household caste/ethnicity category, along with the total number of households and the average household size in each category?

In [37]:
%%sql
WITH household_info AS (
    SELECT
        hd.caste_household,
        COUNT(DISTINCT hd.household_id) AS num_households,
        ROUND(AVG(hd.size_household)) AS avg_household_size
    FROM household_demographics hd
    GROUP BY hd.caste_household
),
building_info AS (
    SELECT
        bd.building_id,
        bs.age_building
    FROM building_damage bd
    JOIN building_structure bs ON bd.building_id = bs.building_id
)
SELECT
    hi.caste_household,
    AVG(bi.age_building) AS avg_building_age,
    hi.num_households,
    hi.avg_household_size
FROM household_info hi
LEFT JOIN id_map im ON hi.caste_household = im.household_id
LEFT JOIN building_info bi ON im.building_id = bi.building_id
GROUP BY hi.caste_household, hi.num_households, hi.avg_household_size;


 * sqlite:///nepal.sqlite
Done.


caste_household,avg_building_age,num_households,avg_household_size
0thers,None,1801,5.0
Badhaee,None,30,5.0
Badi,None,15,5.0
Bangali,None,1,6.0
Baraee,None,172,5.0
Bhote,None,900,4.0
Bin,None,1,3.0
Bote,None,7,4.0
Brahman-Hill,None,39301,5.0
Brahman-Tarai,None,56,4.0


### What is the distribution of damage grades for each type of foundation used in the buildings?

In [38]:
%%sql
SELECT
    bs.foundation_type,
    bd.damage_grade,
    COUNT(*) AS count
FROM building_structure bs
JOIN building_damage bd ON bs.building_id = bd.building_id
GROUP BY bs.foundation_type, bd.damage_grade;


 * sqlite:///nepal.sqlite
Done.


foundation_type,damage_grade,count
Bamboo/Timber,Grade 1,956
Bamboo/Timber,Grade 2,464
Bamboo/Timber,Grade 3,405
Bamboo/Timber,Grade 4,298
Bamboo/Timber,Grade 5,309
Cement-Stone/Brick,Grade 1,1493
Cement-Stone/Brick,Grade 2,1669
Cement-Stone/Brick,Grade 3,1895
Cement-Stone/Brick,Grade 4,1718
Cement-Stone/Brick,Grade 5,1120


### What is the average age of buildings for each district, considering only buildings with severe damage?

In [40]:
%%sql
WITH severe_damage_buildings AS (
    SELECT
        bs.building_id,
        bs.age_building,
        bd.damage_grade,
        id.district_id
    FROM building_structure bs
    JOIN building_damage bd ON bs.building_id = bd.building_id
    JOIN id_map id ON bs.building_id = id.building_id
    WHERE bd.damage_grade = 'Grade 5'
)
SELECT
    district_id,
    ROUND(AVG(age_building)) AS average_age
FROM severe_damage_buildings
GROUP BY district_id;


 * sqlite:///nepal.sqlite
Done.


district_id,average_age
1,19.0
2,23.0
3,36.0
4,29.0
